In [1]:
exec('from __future__ import unicode_literals')

import os
import sys
import random
import json

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../onmt'))
if module_path not in sys.path:
    sys.path.append(module_path)

from itertools import repeat

from onmt.utils.logging import init_logger
from onmt.utils.misc import split_corpus
import onmt.translate.translator as translator

import onmt.opts as opts
from onmt.utils.parse import ArgumentParser
from kp_gen_eval import _get_parser


from nltk.corpus import stopwords
stoplist = stopwords.words('english')

from string import punctuation
import onmt.keyphrase.pke as pke
from nltk.corpus import stopwords

import onmt.keyphrase.kp_inference as kp_inference

/Users/memray/Project/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
import importlib
importlib.reload(kp_inference)
importlib.reload(translator)

<module 'onmt.translate.translator' from '/Users/memray/project/kp/OpenNMT-kpg-release/onmt/translate/translator.py'>

### Load a text (assume current directory is OpenNMT-kpg/notebook/)

In [3]:
data_root_path = '../data/keyphrase/json/duc/duc_test.json'
doc_dicts = []
with open(data_root_path, 'r') as data_file:
    doc_dicts = [json.loads(l) for l in data_file]
        
print('Loaded #(docs)=%d' % (len(doc_dicts)))

Loaded #(docs)=308


##### Sample a paragraph

In [4]:
doc_id = random.randint(0, len(doc_dicts))
doc = doc_dicts[doc_id]
print(doc.keys())
text_to_extract = doc['abstract']
print(doc_id)
print(text_to_extract)


dict_keys(['name', 'title', 'abstract', 'fulltext', 'keywords'])
87
Drugs are now available that can put AIDS patients into remission, and recent advances have made clear that a vaccine to protect against AIDS infection is possible, a panel of AIDS experts said Saturday. At the same time, however, the AIDS epidemic is being followed by a suddenly resurgent epidemic of tuberculosis, the scientists said. ``We have now made demonstrable first steps in inducing remission,'' said William Haseltine of the Dana-Farber Cancer Institute in Boston. ``I think there is evidence that a substantial number of people who would have died are now alive,'' Haseltine said at the annual meeting of the American Association for the Advancement of Science. He cautioned against a premature conclusion that the AIDS epidemic may be coming to an end. A number of treatments for AIDS and AIDS-related infections are available, he said, but many of them are not available to the poor or to developing countries. ``It l

### Supervised Deep Keyphrase Model

In [5]:
parser = _get_parser()
config_path = '../config/translate/config-rnn-keyphrase.yml'
one2one_ckpt_path = '../models/keyphrase/meng17-one2one-kp20k-topmodels/kp20k-meng17-one2one-rnn-BS128-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Covfalse-Contboth-IF1_step_30000.pt'
one2seq_ckpt_path = '../models/keyphrase/meng17-one2seq-kp20k-topmodels/kp20k-meng17-verbatim_append-rnn-BS64-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Reusetrue-Covtrue-PEfalse-Contboth-IF1_step_50000.pt'
opt = parser.parse_args('-config %s' % (config_path))
setattr(opt, 'models', [one2one_ckpt_path])

translator = translator.build_translator(opt, report_score=False)

In [6]:
scores, predictions = translator.translate(
    src=[text_to_extract],
    tgt=None,
    src_dir=opt.src_dir,
    batch_size=opt.batch_size,
    attn_debug=opt.attn_debug,
    opt=opt
)
print('Paragraph:\n\t'+text_to_extract)
print('Top predictions:')
keyphrases = [kp.strip() for kp in predictions[0] if (not kp.lower().strip() in stoplist) and (kp != '<unk>' )]
for kp_id, kp in enumerate(keyphrases[: min(len(keyphrases), 20)]):
    print('\t%d: %s' % (kp_id+1, kp))

/Users/memray/Project/anaconda3/lib/python3.6/site-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


Translating 10/1


/Users/memray/project/kp/OpenNMT-kpg-release/onmt/translate/beam_search.py:226: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1595629444482/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  finished_hyp = self.is_finished[i].nonzero().view(-1)


Total translation time (s): 0.635809
Average translation time (s): 0.635809
Tokens per second: 1.572799
Paragraph:
	Drugs are now available that can put AIDS patients into remission, and recent advances have made clear that a vaccine to protect against AIDS infection is possible, a panel of AIDS experts said Saturday. At the same time, however, the AIDS epidemic is being followed by a suddenly resurgent epidemic of tuberculosis, the scientists said. ``We have now made demonstrable first steps in inducing remission,'' said William Haseltine of the Dana-Farber Cancer Institute in Boston. ``I think there is evidence that a substantial number of people who would have died are now alive,'' Haseltine said at the annual meeting of the American Association for the Advancement of Science. He cautioned against a premature conclusion that the AIDS epidemic may be coming to an end. A number of treatments for AIDS and AIDS-related infections are available, he said, but many of them are not availabl

### PKE models

#### TF-IDF

In [3]:
dataset_name = 'test'
dataset_path = '../data/%s/' % dataset_name
_ = kp_inference.extract_pke(text_to_extract, method='tfidf' , dataset_path=dataset_path,
            df_path=os.path.abspath(dataset_path + '../%s.df.tsv.gz' % dataset_name), top_k=20)

	1: anything (0.0000)
	2: anything that (0.0000)
	3: anything that threatens (0.0000)
	4: that (0.0000)
	5: that threatens (0.0000)
	6: threatens (0.0000)
	7: company (0.0000)
	8: ability (0.0000)
	9: ability to (0.0000)
	10: ability to meet (0.0000)
	11: to meet (0.0000)
	12: to meet its (0.0000)
	13: meet (0.0000)
	14: meet its (0.0000)
	15: meet its target (0.0000)
	16: its (0.0000)
	17: its target (0.0000)
	18: its target or (0.0000)
	19: target (0.0000)
	20: target or (0.0000)


#### YAKE

In [4]:
_ = kp_inference.extract_pke(text_to_extract, method='yake', top_k=20)

	1: called business risk (0.0331)
	2: financial goals (0.0332)
	3: called business (0.0632)
	4: business risk (0.0830)
	5: risk (0.1075)
	6: company (0.1209)
	7: anything (0.1383)
	8: ability (0.1383)
	9: risks may come (0.1415)
	10: business (0.1657)
	11: may (0.1765)
	12: threatens (0.1793)
	13: meet (0.1793)
	14: target (0.1793)
	15: achieve (0.1793)
	16: financial (0.1793)
	17: goals (0.1793)
	18: called (0.1793)
	19: risk management strategy (0.1844)
	20: anything that threatens (0.1851)


#### TextRank

In [5]:
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text_to_extract,
                        language='en_core_web_sm',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)
for kp_id, kp in enumerate(keyphrases):
    print('\t%d: %s (%.4f)' % (kp_id+1, kp[0], kp[1]))

	1: risk management (0.1901)
	2: company head (0.1464)
	3: financial goals (0.1464)
	4: risk (0.0950)
	5: sources (0.0732)
	6: company (0.0732)
